<a href="https://colab.research.google.com/github/nadiajelani/wound-detection/blob/main/Wound_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install tensorflow pillow numpy

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def train_wound_cnn_model(data_dir, save_path="wound_cnn_model.h5"):
    # Data augmentation and normalization
    datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

    # Training and validation data generators
    train_generator = datagen.flow_from_directory(
        data_dir,
        target_size=(128, 128),
        batch_size=32,
        class_mode='binary',
        subset='training'
    )
    val_generator = datagen.flow_from_directory(
        data_dir,
        target_size=(128, 128),
        batch_size=32,
        class_mode='binary',
        subset='validation'
    )

    # Debugging dataset loading
    if train_generator.samples == 0:
        raise ValueError("No training images found. Check your dataset structure and path.")
    if val_generator.samples == 0:
        raise ValueError("No validation images found. Check your dataset structure and path.")

    # CNN Model
    model = Sequential([
        Input(shape=(128, 128, 3)),  # Define input shape explicitly
        Conv2D(32, (3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')  # Binary classification: infected/non-infected
    ])

    # Compile the model
    model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

    # Train the model
    model.fit(
        train_generator,
        validation_data=val_generator,
        epochs=10
    )

    # Save the model
    model.save(save_path)
    print(f"Model saved to {save_path}")

# Train and save the model
train_wound_cnn_model("/content/drive/MyDrive/Colab Notebooks/Jay Jelani/dataset", "/content/drive/MyDrive/Colab Notebooks/Jay Jelani/wound_cnn_model.h5")


Found 162 images belonging to 3 classes.
Found 39 images belonging to 3 classes.
Epoch 1/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 9s 1s/step - accuracy: 0.3817 - loss: -3.5047 - val_accuracy: 0.4359 - val_loss: -32.4419
Epoch 2/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 5s 651ms/step - accuracy: 0.4582 - loss: -67.7083 - val_accuracy: 0.4359 - val_loss: -220.3801
Epoch 3/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 6s 789ms/step - accuracy: 0.4421 - loss: -379.0518 - val_accuracy: 0.4359 - val_loss: -810.3500
Epoch 4/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 9s 649ms/step - accuracy: 0.4331 - loss: -1223.5098 - val_accuracy: 0.4359 - val_loss: -2244.0977
Epoch 5/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 6s 916ms/step - accuracy: 0.4301 - loss: -3257.8750 - val_accuracy: 0.4359 - val_loss: -5454.5815
Epoch 6/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 9s 637ms/step - accuracy: 0.4130 - loss: -7242.2036 - val_accuracy: 0.4359 - val_loss: -10755.4902
Epoch 7/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 7s 1s/step - accuracy: 0.4386 - loss: -14693.1406 - val_accuracy: 0.4359 - val_loss: -19976

Model saved to /content/drive/MyDrive/Colab Notebooks/Jay Jelani/wound_cnn_model.h5


In [ ]:
import os
import subprocess
import numpy as np
from PIL import Image, ImageEnhance, ImageDraw
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.metrics import BinaryAccuracy
from tkinter import Tk, filedialog, Button
from datetime import datetime

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam


In [ ]:
# Load the ResNet50 model for feature extraction
base_model = ResNet50(weights='imagenet')
model = Model(inputs=base_model.input, outputs=base_model.get_layer('avg_pool').output)

# Load the U-Net model for segmentation
try:
    unet_model = load_model('/content/drive/MyDrive/Colab Notebooks/Jay Jelani/unet_model.h5')
except IOError:
    print("Error: U-Net model file could not be loaded. Check the file path.")
    unet_model = None

# Load the CNN model for wound classification and compile it
try:
    cnn_model = load_model('/content/drive/MyDrive/Colab Notebooks/Jay Jelani/wound_cnn_model.h5')
    cnn_model.compile(loss=BinaryCrossentropy(), metrics=[BinaryAccuracy()])
except IOError:
    print("Error: CNN model file could not be loaded. Check the file path.")
    cnn_model = None

# Global variables to control capturing and directory path
is_capturing = True
save_directory = ""
latest_image_path = ""


In [ ]:
# Load the CNN model for wound classification and compile it
try:
    cnn_model = load_model('/content/drive/MyDrive/Colab Notebooks/Jay Jelani/wound_cnn_model.h5')
    cnn_model.compile(optimizer='adam', loss=BinaryCrossentropy(), metrics=[BinaryAccuracy()])
except IOError:
    print("Error: CNN model file could not be loaded. Check the file path.")
    cnn_model = None


In [ ]:
# Load the U-Net model for segmentation
try:
    unet_model = load_model('/content/drive/MyDrive/Colab Notebooks/Jay Jelani/unet_model.h5')
    unet_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
except IOError:
    print("Error: U-Net model file could not be loaded. Check the file path.")
    unet_model = None


In [ ]:
import os
from tensorflow.keras.models import load_model
from skimage.io import imread
from skimage.transform import resize
import numpy as np
from PIL import Image

# Load your U-Net model
unet_model = load_model('/content/drive/MyDrive/Colab Notebooks/Jay Jelani/unet_model.h5')

# Define paths
input_folder = '/content/drive/MyDrive/Colab Notebooks/Jay Jelani/dataset/infected'
output_folder = '/content/drive/MyDrive/Colab Notebooks/Jay Jelani/dataset/masks'

# Ensure output folder exists
os.makedirs(output_folder, exist_ok=True)

# Function to create and save masks
def create_masks(input_folder, output_folder, model, img_size=(128, 128)):
    for file_name in os.listdir(input_folder):
        if file_name.endswith(('.jpg', '.png', '.jpeg')):  # Check for valid image formats
            image_path = os.path.join(input_folder, file_name)
            save_path = os.path.join(output_folder, file_name)

            # Load and preprocess image
            image = imread(image_path)
            image_resized = resize(image, img_size)  # Resize to model's input size
            image_preprocessed = np.expand_dims(image_resized / 255.0, axis=0)  # Normalize and add batch dimension

            # Predict mask
            predicted_mask = model.predict(image_preprocessed)[0]
            binary_mask = (predicted_mask > 0.5).astype(np.uint8)  # Convert to binary mask

            # Save mask
            mask_image = Image.fromarray(binary_mask.squeeze() * 255)  # Scale to [0, 255]
            mask_image.save(save_path)

            print(f"Mask created and saved: {save_path}")

# Create masks for all images
create_masks(input_folder, output_folder, unet_model)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 555ms/step
Mask created and saved: /content/drive/MyDrive/Colab Notebooks/Jay Jelani/dataset/masks/c9514185a0779a67ab879ba57b1b4330_0.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 212ms/step
Mask created and saved: /content/drive/MyDrive/Colab Notebooks/Jay Jelani/dataset/masks/bff9ae20248b28c7f3d7021c1a3a03aa_0.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 255ms/step
Mask created and saved: /content/drive/MyDrive/Colab Notebooks/Jay Jelani/dataset/masks/baba5451a41a555f6ea4355b4c53db59_0.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 200ms/step
Mask created and saved: /content/drive/MyDrive/Colab Notebooks/Jay Jelani/dataset/masks/c13b32121da4b50bcdca469a79b5e7e0_0.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 202ms/step
Mask created and saved: /content/drive/MyDrive/Colab Notebooks/Jay Jelani/dataset/masks/c2313e2f7edbaa2a6286e3284fff445a_0.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 203ms/step
Mask created and saved: /content/drive/MyDrive/Colab Notebooks/Jay Jelani/dataset/masks/ba4755da013537410deae3f8386d08ba_1.png
1/1 

In [10]:
import os
import numpy as np

# Define paths
input_folder = '/content/drive/MyDrive/Colab Notebooks/Jay Jelani/dataset'

def train_cnn_model(input_folder):
    # Check if the input folder exists
    if not os.path.exists(input_folder):
        raise ValueError(f"Input folder '{input_folder}' does not exist. Please check the path.")

    # Check subdirectories for classes
    classes = os.listdir(input_folder)
    if len(classes) < 2:
        raise ValueError(f"Input folder '{input_folder}' should contain at least two subdirectories (e.g., 'infected' and 'non_infected'). Found: {classes}")

    # Data augmentation and data generator
    datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

    try:
        train_gen = datagen.flow_from_directory(
            input_folder,
            target_size=(128, 128),
            batch_size=32,
            class_mode='binary',
            subset='training'
        )

        val_gen = datagen.flow_from_directory(
            input_folder,
            target_size=(128, 128),
            batch_size=32,
            class_mode='binary',
            subset='validation'
        )

        # Check if data generators loaded samples
        if train_gen.samples == 0:
            raise ValueError(f"No training images found in '{input_folder}'. Please check the folder structure.")
        if val_gen.samples == 0:
            raise ValueError(f"No validation images found in '{input_folder}'. Please check the folder structure.")
    except Exception as e:
        raise RuntimeError(f"Error initializing data generators: {e}")

    # Define the CNN model
    cnn_model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
        MaxPooling2D(pool_size=(2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')  # Binary classification
    ])

    # Compile the CNN model
    cnn_model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

    # Train the CNN model
    try:
        cnn_model.fit(
            train_gen,
            validation_data=val_gen,
            epochs=20,
            batch_size=32
        )
    except Exception as e:
        raise RuntimeError(f"Error during training: {e}")

    # Save the model
    try:
        cnn_model.save('/content/drive/MyDrive/Colab Notebooks/Jay Jelani/wound_cnn_model_trained.h5')
        print("CNN model saved as 'wound_cnn_model_trained.h5'")
    except Exception as e:
        raise IOError(f"Error saving model: {e}")

# Call the function to train the CNN model
if __name__ == "__main__":
    try:
        train_cnn_model(input_folder)
    except Exception as e:
        print(f"Error: {e}")


ValueError: Input folder '/content/drive/MyDrive/Colab Notebooks/Jay Jelani/dataset' does not exist. Please check the path.

In [3]:
import os
import logging
import numpy as np
import cv2
from PIL import Image, ImageEnhance, ImageDraw
from tensorflow.keras.models import load_model
from datetime import datetime

# Configure logging
logging.basicConfig(level=logging.INFO)

# Load the trained U-Net model
try:
    unet_model = load_model('/content/drive/MyDrive/Colab Notebooks/Jay Jelani/unet_model.h5')
    logging.info("U-Net model loaded successfully.")
except Exception as e:
    logging.error(f"Failed to load U-Net model: {e}")
    unet_model = None

# Load the trained CNN model
try:
    cnn_model = load_model('/content/drive/MyDrive/Colab Notebooks/Jay Jelani/wound_cnn_model.h5')
    logging.info("CNN model loaded successfully.")
except Exception as e:
    logging.error(f"Failed to load CNN model: {e}")
    cnn_model = None


# Image Capture
def capture_image(save_directory):
    """
    Captures an image using the webcam and saves it to the specified directory.
    """
    try:
        if not os.path.exists(save_directory):
            os.makedirs(save_directory)

        # Start the webcam
        cap = cv2.VideoCapture(0)  # 0 for default webcam
        if not cap.isOpened():
            raise IOError("Cannot access webcam")

        logging.info("Press 'c' to capture the image or 'q' to quit.")
        while True:
            ret, frame = cap.read()
            if not ret:
                logging.error("Failed to capture frame from webcam.")
                break

            # Show live feed
            cv2.imshow("Webcam - Press 'c' to capture", frame)

            # Check for keypress
            key = cv2.waitKey(1) & 0xFF
            if key == ord('c'):  # Capture image
                image_path = os.path.join(save_directory, 'captured_image.jpg')
                cv2.imwrite(image_path, frame)
                logging.info(f"Image captured and saved to {image_path}")
                cap.release()
                cv2.destroyAllWindows()
                return image_path
            elif key == ord('q'):  # Quit
                logging.info("Capture cancelled.")
                cap.release()
                cv2.destroyAllWindows()
                return None
    except Exception as e:
        logging.error(f"Error during image capture: {e}")
        return None


# Image Enhancement
def enhance_image(image_path):
    """
    Enhances the detail of an image and saves it back to the same path.
    """
    try:
        img = Image.open(image_path)
        enhancer = ImageEnhance.Detail(img)
        img_enhanced = enhancer.enhance(1.5)  # Adjust enhancement level
        img_enhanced.save(image_path)
        logging.info(f"Image enhanced and saved to {image_path}")
    except Exception as e:
        logging.error(f"Error enhancing image: {e}")


# Segmentation and Labeling
def segment_and_label_wound(image_path):
    """
    Segments and labels wound areas on an image.
    """
    img = Image.open(image_path)
    mask_image = segment_image_with_unet(image_path)

    if mask_image is None:
        logging.error("Segmentation could not be completed.")
        return None, None

    # Resize mask to original image size
    mask_resized = mask_image.resize(img.size)
    mask_np = np.array(mask_resized)

    # Convert mask to binary and find contours
    _, binary_mask = cv2.threshold(mask_np, 127, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(binary_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    labeled_image = img.copy()
    draw = ImageDraw.Draw(labeled_image)
    wound_areas = []

    # Process each contour
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        draw.rectangle([(x, y), (x + w, y + h)], outline="green", width=2)  # Draw bounding box

        # Classify wound area
        label, confidence = classify_wound_area(img, (x, y, w, h))
        draw.text((x, y - 10), f"{label} ({confidence:.2f})", fill="green")
        wound_areas.append((x, y, w, h, label, confidence))

    # Save labeled image
    labeled_image_path = image_path.replace('.jpg', '_labeled.jpg')
    labeled_image.save(labeled_image_path)
    logging.info(f"Labeled image saved to {labeled_image_path}")
    return labeled_image_path, wound_areas


# Full Processing Pipeline
def process_image(image_path):
    """
    Full pipeline to enhance an image, segment it, classify wounds, and save results.
    """
    enhance_image(image_path)
    labeled_image_path, wound_areas = segment_and_label_wound(image_path)
    if wound_areas:
        export_report(image_path, wound_areas)


# Export Report
def export_report(image_path, wound_areas):
    """
    Exports a detailed wound analysis report.
    """
    report_path = image_path.replace('.jpg', '_report.txt')
    try:
        with open(report_path, 'w') as report:
            report.write(f"Wound Analysis Report - {datetime.now()}\n")
            report.write(f"Image Path: {image_path}\n")
            report.write("Detected Wound Areas (in pixels):\n")
            for area in wound_areas:
                x, y, w, h, label, confidence = area
                report.write(f"- Position: ({x}, {y}), Size: ({w}x{h}), Label: {label}, Confidence: {confidence:.2f}\n")
            total_area = sum(area[2] * area[3] for area in wound_areas)
            report.write(f"Total Wound Area: {total_area} pixels\n")
        logging.info(f"Report saved to {report_path}")
    except Exception as e:
        logging.error(f"Error generating report: {e}")


# Main Execution
if __name__ == "__main__":
    save_directory = '/path/to/save/directory'  # Replace with your directory path
    captured_image_path = capture_image(save_directory)

    if captured_image_path:
        process_image(captured_image_path)
    else:
        logging.info("No image captured. Exiting.")
